In [1]:
import numpy as np
import cv2
import networkx as nx
import math
import matplotlib.pyplot as plt
import torch
import json
import os
import glob

from PIL import Image
from transformers import pipeline
from our_utils import *

video_name = 'bulk_process_1/Bilbao_Spain - Bilbao Spain walking tour [Bilbao Spain walking tour] - [z6-M9g6ayLE]'

print(video_name)

pattern = glob.escape(video_name) + "/*.json"
file = glob.glob(pattern)
file = file[0]

print(file)

with open(f"{file}", "r") as fp:
    data = json.load(fp)
    
cmap = plt.get_cmap("tab20")
colormap = plt.cm.get_cmap("tab20", 20)

N = 20
colors = [cmap(i) for i in range(N)]
palette = (np.array(colors)[:, :3] * 255).astype(np.uint8)

hash_table = {}

device = 'cuda' if torch.cuda.is_available() else 'cpu'

video_path = data['path'] # UPDATED
print(video_path)
cap = cv2.VideoCapture(video_path) # UPDATED
spacing  = data['spacing'] # UPDATED

toProcess = data['frames']

for ind, frame_data in enumerate(toProcess):
    
    frame_id = frame_data['frame_id']
    detections = frame_data['detections']

    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id*spacing) # UPDATED

    ret, image = cap.read()    
    frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
 
    coord3 = np.zeros((1000,image.shape[1],3),dtype=np.uint8)+255
    coord2 = np.zeros((1000,1000,3),dtype=np.uint8)+255
    coord1 = np.zeros((1000,image.shape[1],3),dtype=np.uint8)+255

    G = nx.DiGraph()
    #G.add_edges_from(edges)

    ### UPDATED ####
    
    components = frame_data.get("components_160", [])
    for comp in components:
        for i in range(len(comp)):
            for j in range(i + 1, len(comp)):
                G.add_edge(comp[i], comp[j])
    
    ### UPDATED ####
    
    groups = np.arange(1000)
    for idx, comp in enumerate(components, 1):
        
        key = "-".join(str(c) for c in comp)
        for c in comp:
            groups[c] = idx

        if key not in hash_table:
            hash_table[key]=1
        else:
            hash_table[key]+=1

    
    for int_id, det in enumerate(detections):
        
        x1, y1, x2, y2 = map(int, det['bbox'])
        track_id = det['track_id']
        #print(track_id)
        class_name = 'person'
        label = f"{class_name} ID:{track_id}"
        rgb_tuple = (int(palette[track_id%20][0]),int(palette[track_id%20][1]),int(palette[track_id%20][2]))

        o1_mid = ((x1+x2)//2, (y1+y2)//2) # updated
        d1 = int(det['depth']) # updated
        
        X, Y, Z = det['3D_160FOV']

        h1, w1 = coord1.shape[:2]
        h2, w2 = coord2.shape[:2]
        h3, w3 = coord3.shape[:2]

        center1 = (int((X+2)*100), 1000-int(Z*100)) # updated
        center2 = (int((X+2)*100), int(d1)*4) # updated
        center3 = (int(o1_mid[0]), int(d1)*4) # updated
        
        direction = det['direction']
        offset = 25
        
        plot_coord(coord1, center1, direction, offset, rgb_tuple)
        plot_coord(coord2, center2, direction, offset, rgb_tuple)
        plot_coord(coord3, center3, direction, offset, rgb_tuple)
        
        if class_name == 'person':
            draw_tracking(frame=image, bbox = det['bbox'], label=label, color=rgb_tuple)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    coord1 = cv2.cvtColor(coord1, cv2.COLOR_BGR2RGB)
    coord2 = cv2.cvtColor(coord2, cv2.COLOR_BGR2RGB)
    coord3 = cv2.cvtColor(coord3, cv2.COLOR_BGR2RGB)
    
    
    pos = nx.spring_layout(G, seed=42)
        
    node_colors = [colormap(n%20) for n in G.nodes()]

    fig, ax = plt.subplots(figsize=(4, 4))
    nx.draw(
        G, pos=pos, ax=ax,
        with_labels=True,
        node_color=node_colors,
        edge_color="gray"
    )
    ax.set_axis_off()
    plt.tight_layout()
    
    fig.canvas.draw()
    graph = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    graph = graph.reshape(fig.canvas.get_width_height()[::-1] + (4,))  # 4 channels
    graph = graph[:,:,:3]
    graph = cv2.cvtColor(graph, cv2.COLOR_BGR2RGB)
    plt.close(fig)

    combined_frame1 = np.hstack([cv2.resize(image, (600, 320)), cv2.resize(coord2, (600, 320))])    
    combined_frame2 = np.hstack([cv2.resize(coord3, (600, 320)), cv2.resize(graph, (600, 320))])
    combined_frame = np.vstack([combined_frame1, combined_frame2])

    if True:
        plt.figure(figsize=(36, 8))
        plt.imshow(combined_frame)
        plt.axis("off")
        plt.title(f"Frame {ind}")
        plt.show()

    # save_dir = os.path.join(video_name, "output_frames")
    # os.makedirs(save_dir, exist_ok=True)
    # save_path = os.path.join(save_dir, f"frame_{ind}.jpg")
    # cv2.imwrite(save_path, cv2.cvtColor(combined_frame, cv2.COLOR_RGB2BGR))


    

cap.release()    
print(hash_table)

/gpfs/projects/CascanteBonillaGroup/jmurrugarral/anaconda3/envs/py10-videos/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/gpfs/projects/CascanteBonillaGroup/jmurrugarral/anaconda3/envs/py10-videos/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


bulk_process_1/Bilbao_Spain - Bilbao Spain walking tour [Bilbao Spain walking tour] - [z6-M9g6ayLE]
bulk_process_1/Bilbao_Spain - Bilbao Spain walking tour [Bilbao Spain walking tour] - [z6-M9g6ayLE]/Bilbao_000.json
egocentric_subset/Bilbao_Spain - Bilbao Spain walking tour [Bilbao Spain walking tour] - [z6-M9g6ayLE]/chunk_000.mp4


/tmp/ipykernel_3331999/2402029698.py:29: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = plt.cm.get_cmap("tab20", 20)


{'1-3': 36, '4-5-6-7': 4, '9-10-11-12': 2, '7-8': 1, '4-5-6': 7, '9-10-11': 12, '4-5-6-7-8-13': 1, '4-5-6-7-13': 3, '10-11': 12, '4-5': 8, '6-7': 1, '7-13': 10, '9-11': 10, '4-6': 1, '4-5-7': 8, '4-5-7-13': 8, '32-35': 9, '38-52': 6, '25-27': 3, '30-68': 6, '52-87': 1, '98-102': 14, '94-105': 1, '110-119': 7, '118-156': 1, '118-156-161': 1, '118-127-156-161': 2, '127-134-156-161': 2, '176-178': 1, '178-182': 2, '177-178-182': 1, '176-177': 5, '178-188': 2, '193-194': 1}


In [2]:
posible_groups = sorted(hash_table.items(), key= lambda item:item[1], reverse=True)
print(posible_groups)
k = len(posible_groups)//2

def draw_raw(frame, detection):
    x1, y1, x2, y2 = map(int, detection)
    cv2.rectangle(frame, (x1, y1), (x2, y2), (255,0,0) , 2)
    cv2.putText(frame, 'person', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
    
with open(f"{file}", "r") as fp:
    data = json.load(fp)
    
frames = data['total_frames']
print(frames)
spacing = data['spacing']

for i in range(k):

    print(f'Group {i+1}.')
    if posible_groups[i][1] > 10:
        
        ids = [int(x) for x in posible_groups[i][0].split('-')]
        cap = cv2.VideoCapture(data['path'])
        frame = 0
        c_frames = 0
    
        j= 0 
        while c_frames < (frames-1):
    
            ret, frame = cap.read()#
    
            if c_frames%spacing == 0:
    
                show = False
                for idx in ids:
    
                    #print(j)
                    track_ids = [det['track_id'] for det in data['frames'][j]['detections']]
                    if idx in track_ids:
                        ind = track_ids.index(idx)
                        draw_raw(frame=frame, detection=data['frames'][j]['detections'][ind]['bbox'])
                        show = True
                
                if show:
                    plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
                    plt.show()
                #out.write(frame)
                j+=1
            
            c_frames+=1  
                          
        #out.release()
        cap.release()
        cv2.destroyAllWindows()

[('1-3', 36), ('98-102', 14), ('9-10-11', 12), ('10-11', 12), ('7-13', 10), ('9-11', 10), ('32-35', 9), ('4-5', 8), ('4-5-7', 8), ('4-5-7-13', 8), ('4-5-6', 7), ('110-119', 7), ('38-52', 6), ('30-68', 6), ('176-177', 5), ('4-5-6-7', 4), ('4-5-6-7-13', 3), ('25-27', 3), ('9-10-11-12', 2), ('118-127-156-161', 2), ('127-134-156-161', 2), ('178-182', 2), ('178-188', 2), ('7-8', 1), ('4-5-6-7-8-13', 1), ('6-7', 1), ('4-6', 1), ('52-87', 1), ('94-105', 1), ('118-156', 1), ('118-156-161', 1), ('176-178', 1), ('177-178-182', 1), ('193-194', 1)]
931
Group 1.
The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.
Group 2.
Group 3.
Group 4.
Group 5.
Group 6.
Group 7.
Group 8.
Group 9.
Group 10.
Group 11.
Group 12.
Group 13.
Group 14.
Group 15.
Group 16.
Group 17.
